In [0]:
from pyspark.sql.functions import col
from delta.tables import DeltaTable


In [0]:
df_silver = spark.sql("SELECT * FROM delta.`abfss://silver@projectandstorage.dfs.core.windows.net/carsales`")
display(df_silver)

In [0]:
df_dealer = spark.sql("SELECT * FROM catalog_for_sales.gold_schema.dim_dealer")
df_model = spark.sql("SELECT * FROM catalog_for_sales.gold_schema.dim_model")   
df_branch= spark.sql("select * from catalog_for_sales.gold_schema.dim_branch")
df_date = spark.sql("SELECT * FROM catalog_for_sales.gold_schema.dim_date")


In [0]:
df_fact = df_silver.join(df_dealer, df_silver.Dealer_ID == df_dealer.Dealer_ID, how='left')\
            .join(df_branch, df_silver.Branch_ID == df_branch.Branch_ID, how='left')\
            .join(df_model, df_silver.Model_ID == df_model.Model_ID, how='left')\
            .join(df_date, df_silver.Date_ID == df_date.Date_ID, how='left')\
            .select(df_silver['Revenue'],df_silver['Units_Sold'],df_silver['RevperUnit'],df_branch["dim_branch_key"],df_model["dim_model_key"],df_dealer["dim_dealer_key"],df_date["dim_date_key"])

In [0]:

if spark.catalog.tableExists("catalog_for_sales.gold_schema.fact_model"):
    # Incremental load
    delta_table_obj = DeltaTable.forName(spark, "catalog_for_sales.gold_schema.fact_model")
    delta_table_obj.alias("t").merge(
        df_fact.alias("s"),
        "t.dim_branch_key = s.dim_branch_key AND t.dim_model_key = s.dim_model_key AND t.dim_dealer_key = s.dim_dealer_key AND t.dim_date_key = s.dim_date_key"
    )\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()
else:
    
    df_fact.write.format('delta')\
                .mode('overwrite')\
                .option("path","abfss://gold@projectandstorage.dfs.core.windows.net/fact_model")\
                .saveAsTable("catalog_for_sales.gold_schema.fact_model")

In [0]:
%sql
select  * from delta.`abfss://gold@projectandstorage.dfs.core.windows.net/fact_model`
limit 5;
